# Using ProcGA

## Initialize

In [102]:
using Statistics

In [108]:
include("../src/ProcGA.jl")

Main.ProcGA

In [2]:
using DelimitedFiles

Read process definition table from file.

In [7]:
tbl1 = readdlm("simple_proc.csv", ',')

3×14 Array{Any,2}:
 "m1"   50  1  3  0  1  0  3  2  0  0  2  1  0
 "m2"   50  1  0  0  0  0  3  0  2  0  0  1  2
 "m3"  100  1  3  0  1  0  3  1  0  1  0  0  0

Separate material info and process time definition.

In [ ]:
## material info
mtbl = tbl1[:,1:2]

In [13]:
# proc info
ptbl = tbl1[:,3:end]

3×12 Array{Any,2}:
 1  3  0  1  0  3  2  0  0  2  1  0
 1  0  0  0  0  3  0  2  0  0  1  2
 1  3  0  1  0  3  1  0  1  0  0  0

Process 2 and 6 can be grouped into one.

In [21]:
gtbl = [Dict(:id=>[2;6],:cnt=>150,:timespan=>3)]

1-element Array{Dict{Symbol,Any},1}:
 Dict(:timespan=>3,:id=>[2, 6],:cnt=>150)

Initialize table data.

In [109]:
ProcGA.settable(ptbl, gtbl,mtbl)

true

## sigle gene

create simple gene from process info

In [16]:
gen1 = ProcGA.jobtablebase()

3×32 Array{Int64,2}:
 1  2  2  2  4  6   6   6   7  7  10  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  6  6  6  8  8  11  12  12  0   0     0  0  0  0  0  0  0  0  0  0  0  0
 1  2  2  2  4  6   6   6   7  9   0     0  0  0  0  0  0  0  0  0  0  0  0

this is not well defined due to collision of process.
let it be validated.

In [17]:
ProcGA.validatejob!(gen1)

3×32 Array{Int64,2}:
 1  2  2  2  4  6  6   6   7   7  10  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  6  6  6  8  8  11  12  12   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  2  2  2  0   4   6   6   6     0  0  0  0  0  0  0  0  0  0  0  0

Mutate it to enrich genes.

In [26]:
ProcGA.mutatejob!(gen1,1)

3×32 Array{Int64,2}:
 10  2  2  2  4  6  6  0   7   7  10  …  0  0  0  0  0  0  0  0  0  0  0  0
  0  1  6  6  6  8  8  0  12  12   0     0  0  0  0  0  0  0  0  0  0  0  0
  0  0  1  2  2  2  0  0   6   4   6     0  0  0  0  0  0  0  0  0  0  0  0

In [27]:
# needs to be validated again
ProcGA.validatejob!(gen1)

3×32 Array{Int64,2}:
 1  2  2  2  4  6  6  0   6   7  7  10  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  6  6  6  8  8  0  11  12  0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  2  2  2  0  0   4   6  6   0     0  0  0  0  0  0  0  0  0  0  0  0

Check grouping correctness.

In [28]:
gp1 = ProcGA.grouptable[1]

Dict{Symbol,Any} with 3 entries:
  :timespan => 3
  :id       => [2, 6]
  :cnt      => 150

In [38]:
f(x) = (x in gp1[:id]) ? x : 0
gen1gp = f.(gen1)

3×32 Array{Int64,2}:
 0  2  2  2  0  6  6  0  6  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  6  6  6  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  2  2  2  0  0  0  6  6  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [57]:
(x->(x in gp1[:id]) ? x : 0).(gen1)[:,1:14]

3×14 Array{Int64,2}:
 0  2  2  2  0  6  6  0  6  0  0  0  0  0
 0  0  6  6  6  0  0  0  0  0  0  0  0  0
 0  0  0  2  2  2  0  0  0  6  6  0  0  6

In [41]:
validlength = ProcGA.validlength

validlength (generic function with 1 method)

In [81]:
ProcGA.grpenalty(gen1)

27

## population

Make a population 100% mutating every genes.

In [88]:
popu = ProcGA.initpopulation(200);

In [89]:
popu[1]

3×32 Array{Int64,2}:
 1  2  2  2  4  6  6   6   7   7  10  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  6  6  6  8  8  11  12  12   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  2  2  2  0   4   6   6   6     0  0  0  0  0  0  0  0  0  0  0  0

In [90]:
popu[2]

3×32 Array{Int64,2}:
 0  1  2  2  2  4  6  6   6   7   7  …  0  0  0  0  0  0  0  0  11  0  0  0
 0  0  1  6  6  6  8  8  11  12  12     0  0  0  0  0  0  0  0   0  0  0  0
 0  0  0  1  2  2  2  0   4   6   6     0  0  0  0  0  0  0  0   0  0  0  0

In [91]:
popu[1] == popu[2]

false

Sort them by penalty count.

In [94]:
ProcGA.sortpopulation!(popu);

In [95]:
popu[1]

3×32 Array{Int64,2}:
 1  2  2  2  4  6  6   6   7   7  10  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  6  6  6  8  8  11  12  12   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  2  2  2  0   4   6   6   6     0  0  0  0  0  0  0  0  0  0  0  0

Let inferior genes to die.

In [96]:
ProcGA.survive!(popu);

In [97]:
length(popu)

159

In [98]:
# crossover and mutation
ProcGA.fillgeneration!(popu,200);

In [99]:
length(popu)

200

In [100]:
# make every genes shriked and valid
ProcGA.shrinkjob!.(popu);
ProcGA.validatejob!.(popu);

In [105]:
v = ProcGA.penalty.(popu);

In [106]:
(minimum(v), median(v), maximum(v))

(34, 55.0, 101)

In [107]:
popu[1]

3×32 Array{Int64,2}:
 1  2  2  2  4  6  6   6   7   7  10  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  6  6  6  8  8  11  12  12   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  2  2  2  0   4   6   6   6     0  0  0  0  0  0  0  0  0  0  0  0

Do same routine several times.

In [144]:
ProcGA.sortpopulation!(popu);
ProcGA.survive!(popu);
ProcGA.fillgeneration!(popu,200);
ProcGA.shrinkjob!.(popu);
ProcGA.validatejob!.(popu);
v = ProcGA.penalty.(popu);
(minimum(v), median(v), maximum(v))

(18, 18.0, 33)

In [145]:
popu[1]

3×32 Array{Int64,2}:
 1  0  2  2  2  4  6   6   6   7  7  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  6  6  6  8  8  11  12  12  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1   0   0   2  2  2     0  0  0  0  0  0  0  0  0  0  0  0

## do all at once

In [146]:
# N generations from current status
ProcGA.evolution!(popu,100);

i:10 => (18, 18.0, 19)
i:20 => (18, 18.0, 33)
i:30 => (18, 18.0, 23)
i:40 => (18, 18.0, 26)
i:50 => (18, 18.0, 18)
i:60 => (18, 18.0, 28)
i:70 => (18, 18.0, 28)
i:80 => (18, 18.0, 28)
i:90 => (18, 18.0, 32)
i:100 => (18, 18.0, 28)


In [147]:
popu[1]

3×32 Array{Int64,2}:
 1  0  2  2  2  4  6   6   6   7  7  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  6  6  6  8  8  11  12  12  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1   0   0   2  2  2     0  0  0  0  0  0  0  0  0  0  0  0

In [148]:
ProcGA.clipjob(popu[1])

3×18 Array{Int64,2}:
 1  0  2  2  2  4  6   6   6   7  7  0  0  0  10  10  11  0
 0  1  6  6  6  8  8  11  12  12  0  0  0  0   0   0   0  0
 0  0  0  0  0  0  1   0   0   2  2  2  4  6   6   6   7  9